# Transformer Preliminaries 


## Framing
> GOAL: Create a notebook with proof-of-concepts/examples for each component of the upcoming transformer/Perceiver architecture. 

Objectives: 
 1. Gain familiarity with the code side of transformers.
 2. Create examples of each unfamiliar components of the project. 
 3. Maximize copy-paste-ability. 


## Diverging 
> Things to create prototypes for.

 1. Image/patch extraction. 
 2. 